In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
df=pd.read_csv("./train.csv")

train_x=df['text'][:7000]
train_y=np.array(df['target'][:7000])
val_x=df['text'][7000:]
val_y=np.array(df['target'][7000:])

vocab_size=20000
embedding_dim=64
max_length=120
trunc_type='post'
oov_token="<OOV>"

tokenizer = Tokenizer(vocab_size,oov_token)
tokenizer.fit_on_texts(train_x)
word_index=tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_x)
padded = pad_sequences(sequences, maxlen=max_length,truncating=trunc_type)

test_seq = tokenizer.texts_to_sequences(val_x)
test_padded = pad_sequences(test_seq, maxlen=max_length,truncating=trunc_type)

In [3]:
from keras.regularizers import l1
# instantiate regularizer
reg = l1(0.01)

Using TensorFlow backend.


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,kernel_initializer='glorot_normal',activity_regularizer=reg,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,kernel_initializer='glorot_normal',activity_regularizer=reg)),
    tf.keras.layers.Dense(32,activation='relu',kernel_initializer='glorot_normal',activity_regularizer=reg),
    tf.keras.layers.Dense(8,activation='relu',kernel_initializer='glorot_normal',activity_regularizer=reg),
    tf.keras.layers.Dense(1,activation='sigmoid',kernel_initializer='glorot_normal',activity_regularizer=reg)
])

In [10]:
num_epochs=5

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

model.fit(padded,train_y,epochs=num_epochs,validation_data=(test_padded,val_y),batch_size=32)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 64)           1280000   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 120, 128)          66048     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_8 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 9         
Total params: 1,617,713
Trainable params: 1,617,713
Non-trainable params: 0
____________________________________________

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'],color='r')
#plt.plot(history.history['loss'],color='r')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('model accuracy')
plt.legend(['train', 'loss'], loc='upper left')